In [ ]:
# 아파트넘버별로 각 행수가 10개이상인 아파트만 추출
ratings_by_date_y = df_jjnam.groupby('apt_no').size()
ratings_by_date_y = ratings_by_date_y.index[ratings_by_date_y>=10]
ratings_by_date_y = ratings_by_date_y.to_list()
len(ratings_by_date_y)

qwe = df_jjnam[df_jjnam['apt_no'].isin(ratings_by_date_y)==True]
qwe['apt_no'] = qwe['apt_no'].astype('float')
qwe['apt_no'] = qwe['apt_no'].astype('int')
qwe.groupby('apt_no').size()

def MakeDateCusum(df_num):
  re_dict = df_num['date_y'].value_counts().to_dict()
  
  df_num['date_y'] = df_num['date_y'].astype('str')
  daterange = pd.DataFrame(pd.date_range(df_num['date_y'][0], df_num['date_y'][len(df_num['date_y'])-1]), columns=['date_y'])
  daterange['date_y'] = daterange['date_y'].astype('str')

  result1 = pd.merge(daterange, df_num, on='date_y', how='left')
  result1 = result1.reset_index(drop = True)
  result1['apt_no'] = result1['apt_no'].fillna(result1['apt_no'][0])
  result1['cust_no_x'] = result1['cust_no_x'].fillna(result1['cust_no_x'][0])
  result1 = result1.drop_duplicates(['date_y','level_2','cust_no_y','apt_nm_y','apt_addr_y','sys_creation_date_y'], keep='first')
  result1 = result1.sort_values(by=['date_y'], axis=0)
  result1 = result1.reset_index(drop=True)
  result1['date_y'] = result1['date_y'].astype('str')
  df_num['date_y'] = df_num['date_y'].astype('str')
  result1['date_count'] = 0

  ddd = list(set(result1['date_y'].unique()))
  ddd.sort()
  
  dict_key_value = {}
  aaa = 0
  
  for temp in ddd:
    if temp in list(re_dict.keys()):
      aaa += re_dict[temp]
      dict_key_value[temp] = aaa
    else:
      dict_key_value[temp] = aaa
      
    result1['date_count'] = result1['date_y'].apply(lambda x: dict_key_value[x])
    return result1

apt_name = [i for i in qwe['apt_no'].unique()]

for i in range(len(apt_name)):
  apt_name[i] = int(float(apt_name[i]))
len(apt_name)

# 아파트넘버별 테이블 추출 및 함수 MakeDateCusum & 아파트넘버별 전체 테이블 합치기
columns = qwe.columns
final = pd.DataFrame(columns=columns)
for name in apt_name:
  globals()['df_{}'.format(name)] = qwe.loc[qwe['apt_no'].isin([name])] # 각 아파트넘버별로 테이블 추출 : 총 239개
  globals()['df_{}'.format(name)] = globals()['df_{}'.format(name)].reset_index(drop=True) # 인덱스 리셋
  globals()['df_{}'.format(name)] = MakeDateCusum(globals()['df_{}'.format(name)]) # 날짜별 누적합 함수 돌리기
  final = final.append(globals()['df_{}'.format(name)], ignore_index=True) # 아파트넘버별 전체 테이블 합치기
  
final